<a href="https://colab.research.google.com/github/fmarcelneves/Twitter-Network-Analytics/blob/main/Machine_learning_model_construction_Felipe_M_Neves_political_theme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Portuguese Tweets for Sentiment Analysis using nltk and sklearn</h1>

## Install package

In [1]:
!pip install xgboost

## Load packages

In [2]:
import os
import pandas as pd
import numpy as np
import random
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
# from nltk.tokenize import sent_tokenize (Tokenization)
from nltk.probability import FreqDist
from nltk.metrics import ConfusionMatrix
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
import  xgboost as xgb

In [3]:
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Load and concatenate dataset

<h4>Load and prepare</h4>

In [4]:
TweetsNeutralHash = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsNeutralHash.csv'
TweetsNeutralNews = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsNeutralNews.csv'
TweetsWithTheme = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/TweetsWithTheme.csv'
Train3Classes = 'https://raw.githubusercontent.com/fmarcelneves/Twitter-Network-Analytics/main/Train3Classes.csv'

In [5]:
NeutralHash = pd.read_csv(TweetsNeutralHash, encoding='utf-8')
NeutralNews = pd.read_csv(TweetsNeutralNews, encoding='utf-8')
Political_Neg_Pos = pd.read_csv(TweetsWithTheme, encoding='utf-8')
Classes = pd.read_csv(Train3Classes, encoding='utf-8', sep=';')

In [6]:
NeutralHash  = NeutralHash[['tweet_text', 'sentiment']]
NeutralHash.columns = ['texto', 'classificacao']

In [7]:
NeutralHash.shape

(15727, 2)

In [8]:
NeutralHash = NeutralHash.drop_duplicates().reset_index(drop=True)

In [9]:
NeutralHash.shape

(15515, 2)

In [10]:
NeutralNews  = NeutralNews[['tweet_text', 'sentiment']]
NeutralNews.columns = ['texto', 'classificacao']

In [11]:
NeutralNews.shape

(37556, 2)

In [12]:
NeutralNews = NeutralNews.drop_duplicates().reset_index(drop=True)

In [13]:
NeutralNews.shape

(37509, 2)

In [14]:
Political_Neg_Pos  = Political_Neg_Pos[['tweet_text', 'sentiment']]
Political_Neg_Pos.columns = ['texto', 'classificacao']

In [15]:
Political_Neg_Pos.shape

(61591, 2)

In [16]:
Political_Neg_Pos = Political_Neg_Pos.drop_duplicates().reset_index(drop=True)

In [17]:
Political_Neg_Pos.shape

(61283, 2)

In [18]:
NeutralHash = NeutralHash.sample(8000) #14000 #8000

In [19]:
NeutralHash.shape

(8000, 2)

In [20]:
NeutralNews = NeutralNews.sample(20000) #14000 #20000

In [21]:
NeutralNews.shape

(20000, 2)

In [22]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].shape

(32566, 2)

In [23]:
Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].shape

(28717, 2)

In [24]:
Political_Pos = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Positivo'].sample(20000) #18000 #20000 #14000

In [25]:
Political_Pos.shape

(20000, 2)

In [26]:
Classes_Pos  = Classes[['tweet_text', 'sentiment']]
Classes_Pos.columns = ['texto', 'classificacao']

In [27]:
Classes_Pos['classificacao'] = Classes_Pos['classificacao'].map({0: 'Negativo', 1: 'Positivo', 2: 'Neutro'})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
Classes_Pos = Classes_Pos.loc[Classes_Pos['classificacao'] == 'Positivo'].sample(8000) #10000 #14000 #8000

In [29]:
Classes_Pos.shape

(8000, 2)

In [30]:
Political_Neg = Political_Neg_Pos.loc[Political_Neg_Pos['classificacao'] == 'Negativo'].sample(28000)

In [31]:
Political_Neg.shape

(28000, 2)

In [32]:
dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, Classes_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
dataset

,texto,classificacao
0,"N adianta, tudo oq vc faz um dia volta pra vc ...",Neutro
1,A disputa pelos talentos na era digital #educa...,Neutro
2,Contribuinte que está no 3º lote do IR 2018 re...,Neutro
3,4 Dicas para otimizar o sistema organizacional...,Neutro
4,#trabalho Port. 22/2018: Reajusta o Valor-piso...,Neutro
...,...,...
83680,@Chuchuziin_ pse veiiii eu quero o ciro :((,Negativo
83681,"@LisetePSOL Que vergonha, uma senhora de idade...",Negativo
83682,@cirogomes Alguém sabe se o Ciro tem alguma pr...,Negativo
83683,A/C @TSEjusbr @MPF_PGR @MPF_eleitoral @policia...,Negativo


In [33]:
dataset.shape

(83685, 2)

In [34]:
#dataset = pd.concat([NeutralHash, NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

In [35]:
#dataset = pd.concat([NeutralNews, Political_Pos, Political_Neg]).drop_duplicates().reset_index(drop=True)
#dataset

## Exploratory Analysis

In [36]:
dataset.sample(5)

,texto,classificacao
55599,"@SamuelWDyck @AMenteDoDidi não duvido disso, s...",Positivo
20915,SP: Doria afirma que vai a todos os debates e ...,Neutro
31623,@perriezandoo @guilhermebubble @paulakpta @out...,Positivo
73340,"Como assim casa dos seus pais, seus pais não e...",Negativo
8480,Aliados avaliam que carta de FH não ajudou Alc...,Neutro


In [37]:
series = dataset['classificacao'].value_counts()
series

Negativo    28000
Neutro      27989
Positivo    27696
Name: classificacao, dtype: int64

In [38]:
%%capture
!pip install plotly --upgrade
from plotly import __version__
print (__version__)
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly as py
#import plotly.figure_factory as ff
from plotly import tools
#import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
import plotly.io as pio
pio.templates.default = "plotly_white"

In [39]:
pd.options.plotting.backend = "plotly"


In [40]:
series.plot(kind='bar', title='Número de tweets classificados para cada sentimento considerado', 
labels={"value": "Frequências", "index": "Sentimentos"}, color = ['Negativo', 'Neutro', 'Positivo'], 
color_discrete_sequence =['#EF553B', '#00CC96', '#636EFA'])

## Text Preprocessing

  Noise Removal
    <ul>
      <li>Remove urls from text (http(s), www)</li>
      <li>Remove a regex pattern (hashtags, notations)</li>
      <li>Prepare text, replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon tags</li>
  <li>Tokenize text with nltk</li>
  <li>Object Standardization</li>
  <li>Remove stopwords (noise removal and dimensionality reduction)</li>
  <li>Lexicon Normalization</li>
    <ul>
      <li>Lemmatization (not implemented...)</li>
      <li>Stemming (dimensionality reduction)</li>
    </ul>
</ul> 

<h4>Examples of emoticons</h4>

<h4>Noise Removal</h4>

Remove urls from text (http(s), www)

In [41]:
def _remove_url(data):
    ls = []
    words = ''
    regexp1 = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    regexp2 = re.compile('www?.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    
    for line in data:
        urls = regexp1.findall(line)

        for u in urls:
            line = line.replace(u, ' ')

        urls = regexp2.findall(line)

        for u in urls:
            line = line.replace(u, ' ')
            
        ls.append(line)
    return ls

In [42]:
dataset['texto'] = _remove_url(dataset['texto'])

Remove a regex pattern (hashtags, notations)

In [43]:
def _remove_regex(data, regex_pattern):
    ls = []
    words = ''
    
    for line in data:
        matches = re.finditer(regex_pattern, line)
        
        for m in matches: 
            line = re.sub(m.group().strip(), '', line)

        ls.append(line)

    return ls

In [44]:
# hashtags
regex_pattern = '#[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [45]:
# notations
regex_pattern = '@[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [46]:
# rt
regex_pattern = 'RT[\w]*'
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [47]:
 emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

In [48]:
# emoji pattern 
regex_pattern = emoji_pattern 
dataset['texto'] = _remove_regex(dataset['texto'], regex_pattern)

In [49]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

N adianta, tudo oq vc faz um dia volta pra vc 
A disputa pelos talentos na era digital   . .       
Contribuinte que está no 3º lote do IR 2018 recebe hoje restituição -   
4 Dicas para otimizar o sistema organizacional de uma empresa   
 Port. 22/2018: Reajusta o Valor-piso da Hora de Trabalho de Economia - VHTE pelo IPCA (IBGE).... 


Prepare text, replace emoticons ":)) :) :D :(" to positive_emoticon or negative_emoticon or neutral_emoticon

In [50]:
def _replace_emoticons(data, emoticon_list):
    ls = []

    for line in data:
        for exp in emoticon_list:
            line = line.replace(exp, emoticon_list[exp])

        ls.append(line)

    return ls

In [51]:
emoticon_list = {':))': 'emoticon_positivo', ':)': 'emoticon_positivo', ':D': 'emoticon_positivo', ':(': 'emoticon_negativo', ':((': 'emoticon_negativo', '8)': 'emoticon_neutro'}

In [52]:
dataset['texto']= _replace_emoticons(dataset['texto'], emoticon_list)

In [53]:
# check data
for i in range(0, 5):
    print(dataset['texto'][i])

N adianta, tudo oq vc faz um dia volta pra vc 
A disputa pelos talentos na era digital   . .       
Contribuinte que está no 3º lote do IR 2018 recebe hoje restituição -   
4 Dicas para otimizar o sistema organizacional de uma empresa   
 Port. 22/2018: Reajusta o Valor-piso da Hora de Trabalho de Economia - VHTE pelo IPCA (IBGE).... 


<h5 style="color:red;">Note: We can remove or replace with tags some noisy words or special characters like ✨💖🎈🎉🎊❤️, but we need to evaluate if they are important to express some sentiment</h5>

In [54]:
dataset

,texto,classificacao
0,"N adianta, tudo oq vc faz um dia volta pra vc",Neutro
1,A disputa pelos talentos na era digital . . ...,Neutro
2,Contribuinte que está no 3º lote do IR 2018 re...,Neutro
3,4 Dicas para otimizar o sistema organizacional...,Neutro
4,Port. 22/2018: Reajusta o Valor-piso da Hora ...,Neutro
...,...,...
83680,pse veiiii eu quero o ciro emoticon_negativo(,Negativo
83681,"Que vergonha, uma senhora de idade envolvida ...",Negativo
83682,Alguém sabe se o Ciro tem alguma proposta par...,Negativo
83683,"A/C A máxima: ""Fechar o STF..."", só se apl...",Negativo


In [55]:
nan_value = float("NaN")
dataset.replace("", nan_value, inplace=True)

<h4>Tokenize text with nltk</h4>

In [56]:
def _tokenize_text(data):
    ls = []

    for line in data:
        tokens = wordpunct_tokenize(line)
        ls.append(tokens)

    return ls

In [57]:
dataset_texto_tokens = _tokenize_text(dataset['texto'])

In [58]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['N', 'adianta', ',', 'tudo', 'oq', 'vc', 'faz', 'um', 'dia', 'volta', 'pra', 'vc']
['A', 'disputa', 'pelos', 'talentos', 'na', 'era', 'digital', '.', '.']
['Contribuinte', 'que', 'está', 'no', '3º', 'lote', 'do', 'IR', '2018', 'recebe', 'hoje', 'restituição', '-']
['4', 'Dicas', 'para', 'otimizar', 'o', 'sistema', 'organizacional', 'de', 'uma', 'empresa']
['Port', '.', '22', '/', '2018', ':', 'Reajusta', 'o', 'Valor', '-', 'piso', 'da', 'Hora', 'de', 'Trabalho', 'de', 'Economia', '-', 'VHTE', 'pelo', 'IPCA', '(', 'IBGE', ')....']


In [59]:
#dataset_texto_tokens = list(filter(None, dataset_texto_tokens))

In [60]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['N', 'adianta', ',', 'tudo', 'oq', 'vc', 'faz', 'um', 'dia', 'volta', 'pra', 'vc']
['A', 'disputa', 'pelos', 'talentos', 'na', 'era', 'digital', '.', '.']
['Contribuinte', 'que', 'está', 'no', '3º', 'lote', 'do', 'IR', '2018', 'recebe', 'hoje', 'restituição', '-']
['4', 'Dicas', 'para', 'otimizar', 'o', 'sistema', 'organizacional', 'de', 'uma', 'empresa']
['Port', '.', '22', '/', '2018', ':', 'Reajusta', 'o', 'Valor', '-', 'piso', 'da', 'Hora', 'de', 'Trabalho', 'de', 'Economia', '-', 'VHTE', 'pelo', 'IPCA', '(', 'IBGE', ')....']


<h4>Object Standardization</h4>

In [61]:
def _apply_standardization(tokens, std_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() in std_list:
                word = std_list[word.lower()]
                
            new_tokens.append(word) 
            
        ls.append(new_tokens)

    return ls

In [62]:
# create your own list
std_list = {'eh': 'é', 'vc': 'você', 'vcs': 'vocês','tb': 'também', 'tbm': 'também', 'obg': 'obrigado', 'gnt': 'gente', 'q': 'que', 'n': 'não', 'cmg': 'comigo', 'p': 'para', 'ta': 'está', 'to': 'estou', 'vdd': 'verdade'}

In [63]:
dataset_texto_tokens = _apply_standardization(dataset_texto_tokens, std_list)

In [64]:
# check data
print(dataset_texto_tokens[4], dataset_texto_tokens[1008])

['Port', '.', '22', '/', '2018', ':', 'Reajusta', 'o', 'Valor', '-', 'piso', 'da', 'Hora', 'de', 'Trabalho', 'de', 'Economia', '-', 'VHTE', 'pelo', 'IPCA', '(', 'IBGE', ')....'] ['Veja', 'o', 'que', 'é', 'ou', 'nas', 'falas', 'dos', 'candidatos', 'ao', 'governo', 'de', 'SP']


<h4>Remove stopwords (noise removal and dimensionality reduction)</h4>

Remove stopwords

In [65]:
def _remove_stopwords(tokens, stopword_list):
    ls = []

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            if word.lower() not in stopword_list:
                new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [66]:
stopword_list = []

In [67]:
# get nltk portuguese stopwords
nltk_stopwords = nltk.corpus.stopwords.words('portuguese')

In [68]:
# You can also add stopwords manually instead of loading from the database. Generally, we add stopwords that belong to this context.
stopword_list.append('é')
stopword_list.append('vou')
stopword_list.append('que')
stopword_list.append('tão')
stopword_list.append('...')
stopword_list.append('«')
stopword_list.append('➔')
stopword_list.append('|')
stopword_list.append('//')
stopword_list.append('/')
stopword_list.append('||') 
stopword_list.append(':')
stopword_list.append('-')
stopword_list.append(';')
stopword_list.append('»')
stopword_list.append('…')
stopword_list.append(')')
stopword_list.append('(')
stopword_list.append('uai') 
stopword_list.append(']')
stopword_list.append('[')
stopword_list.append('((')
stopword_list.append('))')
stopword_list.append('emoticon_positivo') 
stopword_list.append('emoticon_negativo')
stopword_list.append('emoticon_neutro')
# 'expression from the mineiros (MG/Brazil)'

<h5 style="color:red;">Note: Numbers, some punctuations (.,;:) and special characters such as (_- = &%) are not relevant to express some sentiment, they can be considered noisy words. You can treat them here.</h5>
<h5 style="color:red;">noisy_words = ['.', '?', '!', ':', ',', ';', '(', ')', '-']</h5>
<h5 style="color:red;">Note: ? and ! can be important to express sentiment.</h5>

In [69]:
# join all stopwords
stopword_list.extend(nltk_stopwords)
#stopword_list.extend(custom_stopwords)

In [70]:
# remove duplicate stopwords (unique list)
stopword_list = list(set(stopword_list))

In [71]:
dataset_texto_tokens = _remove_stopwords(dataset_texto_tokens, stopword_list)

In [72]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['adianta', ',', 'tudo', 'oq', 'faz', 'dia', 'volta', 'pra']
['disputa', 'talentos', 'digital', '.', '.']
['Contribuinte', '3º', 'lote', 'IR', '2018', 'recebe', 'hoje', 'restituição']
['4', 'Dicas', 'otimizar', 'sistema', 'organizacional', 'empresa']
['Port', '.', '22', '2018', 'Reajusta', 'Valor', 'piso', 'Hora', 'Trabalho', 'Economia', 'VHTE', 'IPCA', 'IBGE', ')....']


<h4>Lexicon Normalization</h4>

Lemmatization (not implemented...)

Stemming (dimensionality reduction)

In [73]:
def _apply_stemmer(tokens):
    ls = []
    stemmer = nltk.stem.RSLPStemmer()

    for tk_line in tokens:
        new_tokens = []
        
        for word in tk_line:
            word = str(stemmer.stem(word))
            new_tokens.append(word) 
            
        ls.append(new_tokens)
        
    return ls

In [74]:
nltk.download('rslp')

[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [75]:
dataset_texto_stemm = _apply_stemmer(dataset_texto_tokens)

In [76]:
# check data
for i in range(0, 5):
    print(dataset_texto_tokens[i])

['adianta', ',', 'tudo', 'oq', 'faz', 'dia', 'volta', 'pra']
['disputa', 'talentos', 'digital', '.', '.']
['Contribuinte', '3º', 'lote', 'IR', '2018', 'recebe', 'hoje', 'restituição']
['4', 'Dicas', 'otimizar', 'sistema', 'organizacional', 'empresa']
['Port', '.', '22', '2018', 'Reajusta', 'Valor', 'piso', 'Hora', 'Trabalho', 'Economia', 'VHTE', 'IPCA', 'IBGE', ')....']


<h4>Dataset preparation</h4>
Untokenize text (transform tokenized text into string list)

In [77]:
def _untokenize_text(tokens):
    ls = []

    for tk_line in tokens:
        new_line = ''
        
        for word in tk_line:
            new_line += word + ' '
            
        ls.append(new_line)
        
    return ls

In [78]:
dataset_texto_tokens = _untokenize_text(dataset_texto_tokens)

In [79]:
dataset_texto_stemm = _untokenize_text(dataset_texto_stemm)

In [80]:
# check data
for i in range(0, 5):
    print(dataset_texto_stemm[i])

adiant , tud oq faz dia volt pra 
disput talent digit . . 
contribuint 3º lot ir 2018 receb hoj restitu 
4 dic otimiz sistem organizac empr 
port . 22 2018 reajust val pis hor trabalh econom vht ipc ibg ).... 


## Implementing a Model


In [81]:
X_tweets = np.array(dataset_texto_stemm,  dtype=object)
X_tweets

array(['adiant , tud oq faz dia volt pra ', 'disput talent digit . . ',
       'contribuint 3º lot ir 2018 receb hoj restitu ', ...,
       'alguém sab cir algum propost ajud animal rua ? prote gost sab . li ouv nad sobr ',
       'c máx " fech stf ...", aplic direit val tod petist continu robert requ list aument . tô trist . ',
       'amig , ent vot cir , ach chanc real '], dtype=object)

In [82]:
y_classes = dataset["classificacao"].values
y_classes

array(['Neutro', 'Neutro', 'Neutro', ..., 'Negativo', 'Negativo',
       'Negativo'], dtype=object)

In [83]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neural_network import MLPClassifier

Split data

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X_tweets, y_classes, test_size=0.33, random_state=42)

## Random Forest

In [84]:
model = RandomForestClassifier(random_state=42, n_jobs=-1)

In [93]:
model_RF = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
])

### Random forest model with Hold-out

In [94]:
model_RF.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [95]:
predictions_RF = model_RF.predict(X_test)

In [96]:
print(classification_report(y_test, predictions_RF))

              precision    recall  f1-score   support

    Negativo       0.69      0.77      0.73      9256
      Neutro       0.86      0.81      0.83      9117
    Positivo       0.65      0.61      0.63      9244

    accuracy                           0.73     27617
   macro avg       0.73      0.73      0.73     27617
weighted avg       0.73      0.73      0.73     27617



In [97]:
#for doc, prediction, y in zip(X_test[0:20], predictions_RF[0:20], y_test[0:20]):
#    print('%r => %s [%s]' % (doc, prediction, y))

### Random Forest model with gridsearch and cross-validation

In [ ]:
RF_param_grid = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 200, 500],
    'clf__max_depth': [None, 1, 3],
    'clf__min_samples_split': [5, 10],
    'clf__min_samples_leaf': [1, 3, 4, 5]
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
grid_rf = GridSearchCV(param_grid=RF_param_grid, estimator=model_RF, 
                        scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)

In [ ]:
grid_rf = RandomizedSearchCV(estimator=model_RF, param_distributions=RF_param_grid,  
                        scoring='f1_weighted', cv=3, verbose=2, n_jobs=-1)

In [ ]:
#grid_rf.fit(X_tweets, y_classes)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  9.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              CountVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.int64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                          

In [ ]:
#print("Best parameters found: ", grid_rf.best_params_)

Best parameters found:  {'clf__n_estimators': 500, 'clf__min_samples_split': 10, 'clf__min_samples_leaf': 5, 'clf__max_depth': None, 'clf__criterion': 'gini'}


In [ ]:
model = RandomForestClassifier(n_estimators= 500, min_samples_split= 10, random_state=42, min_samples_leaf= 5, criterion = 'gini', n_jobs=-1)

In [ ]:
model_RF = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
])

In [ ]:
predictions_RF = cross_val_predict(model_RF, X_tweets, y_classes, cv = 5)

In [ ]:
print(classification_report(y_classes, predictions_RF))

              precision    recall  f1-score   support

    Negativo       0.67      0.77      0.72     28000
      Neutro       0.77      0.85      0.81     27993
    Positivo       0.64      0.48      0.55     27674

    accuracy                           0.70     83667
   macro avg       0.70      0.70      0.69     83667
weighted avg       0.70      0.70      0.69     83667



In [ ]:
#for doc, predictions_RF, y in zip(tweets[0:10], predictions_RF[0:10], classes[0:10]):
#    print('%r => %s [%s]' % (doc, predictions_RF, y))

## XGBoost 

In [98]:
model_XGB = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', xgb.XGBClassifier(n_estimators=1000, n_jobs=-1, nthread=-1))
])

##XGBoost with Hold-out

In [99]:
model_XGB.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing

In [100]:
predictions_XGB = model_XGB.predict(X_test)

In [101]:
print(classification_report(y_test, predictions_XGB))

              precision    recall  f1-score   support

    Negativo       0.75      0.68      0.72      9256
      Neutro       0.75      0.88      0.81      9117
    Positivo       0.65      0.60      0.63      9244

    accuracy                           0.72     27617
   macro avg       0.72      0.72      0.72     27617
weighted avg       0.72      0.72      0.72     27617



## Neural Network - Multilayer Perceptron

<img src="https://st2.ning.com/topology/rest/1.0/file/get/1140047167?profile=original" width="500"/>

In [102]:
model_NN = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('pca', TruncatedSVD(n_components=200)), # 100000 linhas por 150 colunas (features)
    ('clf', MLPClassifier(hidden_layer_sizes=(100,50), #2 camadas escondidas sendo a primeira com 100 neurônios e a segunda com 50 neurônios 
                          random_state=42, 
                          activation='relu', #Testar outras funções de ativação. Ex. tanh, entre outras (relu).
                          max_iter=1000,
                          early_stopping=True))
])

### Neural network model with Hold-out

In [103]:
model_NN.fit(X_train, y_train)
predictions_NN = model_NN.predict(X_test)

In [104]:
print(classification_report(y_test, predictions_NN))

              precision    recall  f1-score   support

    Negativo       0.70      0.74      0.72      9256
      Neutro       0.85      0.82      0.84      9117
    Positivo       0.63      0.62      0.63      9244

    accuracy                           0.73     27617
   macro avg       0.73      0.73      0.73     27617
weighted avg       0.73      0.73      0.73     27617



In [105]:
for doc, prediction, y in zip(X_test[0:20], predictions_NN[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'saib nazist ode jud jerusalém caracterís esquerdopat . além diss esquerd ador corrupt , pedófil , estupr marg . enquadr ness tim ? pesquis amig lulal merd mesm cois :-) ' => Positivo [Positivo]
'príncip harry complet 34 ano receb homen intern ' => Neutro [Neutro]
'blizzard problem overwatch , diz licenç jog tav jog ate ont boa , tod bug compr jog mó feliz mei deix entr ' => Positivo [Negativo]
'oi mig ' => Positivo [Positivo]
'imagin iri ter gent centr quer ir pro lad bolsonar . ' => Negativo [Positivo]
'ali marin tem alv dobr bolsonar daciol debat . ' => Neutro [Neutro]
'aha !!! import pro irm irm vai sent bem famíl lá ^^ ' => Positivo [Positivo]
'big broth ingl políci vê 11 mil hor víde ach suspeit envenen ex espiã . ' => Neutro [Neutro]
'agor sim united par cicl posi watford som prim derrot nest ediç premi leag ' => Positivo [Positivo]
'post elogi manu aqu . ler post , vai entend toler ser zer pra tip açã . ele terr . candidat pres . atitud infeliz . :-( ' => Negativo [Negativo]
'c

## SVM model

In [140]:
model_SVC = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(random_state=42)),
])

## SVM model with Hold out

In [ ]:
model_SVC.fit(X_train, y_train)
predictions_SVC = model_SVC.predict(X_test)

In [139]:
print(classification_report(y_test, predictions_SVC))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



              precision    recall  f1-score   support

    Negativo       0.00      0.00      0.00      9813
      Neutro       0.33      1.00      0.50      9664
    Positivo       0.00      0.00      0.00      9813

    accuracy                           0.33     29290
   macro avg       0.11      0.33      0.17     29290
weighted avg       0.11      0.33      0.16     29290



In [110]:
for doc, prediction, y in zip(X_test[0:20], predictions_SVC[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'saib nazist ode jud jerusalém caracterís esquerdopat . além diss esquerd ador corrupt , pedófil , estupr marg . enquadr ness tim ? pesquis amig lulal merd mesm cois :-) ' => Positivo [Positivo]
'príncip harry complet 34 ano receb homen intern ' => Neutro [Neutro]
'blizzard problem overwatch , diz licenç jog tav jog ate ont boa , tod bug compr jog mó feliz mei deix entr ' => Positivo [Negativo]
'oi mig ' => Positivo [Positivo]
'imagin iri ter gent centr quer ir pro lad bolsonar . ' => Negativo [Positivo]
'ali marin tem alv dobr bolsonar daciol debat . ' => Negativo [Neutro]
'aha !!! import pro irm irm vai sent bem famíl lá ^^ ' => Positivo [Positivo]
'big broth ingl políci vê 11 mil hor víde ach suspeit envenen ex espiã . ' => Neutro [Neutro]
'agor sim united par cicl posi watford som prim derrot nest ediç premi leag ' => Positivo [Positivo]
'post elogi manu aqu . ler post , vai entend toler ser zer pra tip açã . ele terr . candidat pres . atitud infeliz . :-( ' => Negativo [Negativo]


## Logistic Regression

In [122]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

## Logistic Regression with Hold Out

In [123]:
model_LR.fit(X_train, y_train)
predictions_LR = model_LR.predict(X_test)

In [119]:
X_train.shape, X_test.shape

((54395,), (29290,))

In [124]:
print(classification_report(y_test, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.73      0.77      0.75      9813
      Neutro       0.85      0.86      0.86      9664
    Positivo       0.69      0.65      0.67      9813

    accuracy                           0.76     29290
   macro avg       0.76      0.76      0.76     29290
weighted avg       0.76      0.76      0.76     29290



In [125]:
for doc, prediction, y in zip(X_test[0:20], predictions_LR[0:20], y_test[0:20]):
    print('%r => %s [%s]' % (doc, prediction, y))

'saib nazist ode jud jerusalém caracterís esquerdopat . além diss esquerd ador corrupt , pedófil , estupr marg . enquadr ness tim ? pesquis amig lulal merd mesm cois :-) ' => Positivo [Positivo]
'príncip harry complet 34 ano receb homen intern ' => Neutro [Neutro]
'blizzard problem overwatch , diz licenç jog tav jog ate ont boa , tod bug compr jog mó feliz mei deix entr ' => Positivo [Negativo]
'oi mig ' => Positivo [Positivo]
'imagin iri ter gent centr quer ir pro lad bolsonar . ' => Negativo [Positivo]
'ali marin tem alv dobr bolsonar daciol debat . ' => Negativo [Neutro]
'aha !!! import pro irm irm vai sent bem famíl lá ^^ ' => Positivo [Positivo]
'big broth ingl políci vê 11 mil hor víde ach suspeit envenen ex espiã . ' => Neutro [Neutro]
'agor sim united par cicl posi watford som prim derrot nest ediç premi leag ' => Positivo [Positivo]
'post elogi manu aqu . ler post , vai entend toler ser zer pra tip açã . ele terr . candidat pres . atitud infeliz . :-( ' => Negativo [Negativo]


## Logistic Regression with cross-validation and gridsearch 
Doing CV and gridsearch on the model with the best performance with the default settings 

In [126]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, multi_class='multinomial', max_iter=500, n_jobs=-1)),
])

In [127]:
param_grid_LR = {
    'clf__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'clf__penalty': ['l1', 'l2', 'elasticnet', 'none']
}

In [128]:
grid_LR = GridSearchCV(param_grid=param_grid_LR, estimator=model_LR, 
                        scoring='f1_weighted', cv=5, verbose=1, n_jobs=-1)

In [ ]:
#grid_LR.fit(X_tweets, y_classes)

In [ ]:
#print(grid_LR.best_params_)

In [129]:
model = LogisticRegression(random_state=42, multi_class='multinomial', max_iter=100, n_jobs=-1, solver= 'newton-cg')

In [130]:
model_LR = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
])

In [131]:
predictions_LR = cross_val_predict(model_LR, X_tweets, y_classes, cv = 5)

In [132]:
print(classification_report(y_classes, predictions_LR))

              precision    recall  f1-score   support

    Negativo       0.72      0.78      0.75     28000
      Neutro       0.85      0.85      0.85     27989
    Positivo       0.67      0.63      0.65     27696

    accuracy                           0.75     83685
   macro avg       0.75      0.75      0.75     83685
weighted avg       0.75      0.75      0.75     83685



In [ ]:
#for doc, predictions_RF, y in zip(tweets[0:10], predictions_RF[0:10], classes[0:10]):
#    print('%r => %s [%s]' % (doc, predictions_RF, y))

## Exporting models

In [ ]:
import pickle

In [ ]:
#filename = 'LR_model.sav'
pickle.dump(model_LR, open('LR_model_we', 'wb'))

In [ ]:
model_LR.predict(X_test)

In [ ]:
predictions = loaded_model.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))